### Initialization

In [1]:
# Importing libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import gc
from PIL import Image
import os

# Scikit-Learn Tools
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Tensorflow Tools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

In [2]:
# Checking if Tensorflow is running on GPU
print(f"Built With CUDA: { tf.test.is_built_with_cuda() }")
print(f"GPU Available: { tf.test.is_gpu_available() }")
print(f"GPU Device Name: { tf.test.gpu_device_name() }")

Built With CUDA: True
GPU Available: True
GPU Device Name: /device:GPU:0


### Loading Image Data

In [3]:
# Setting up empty lists for data
imageLabels, imageData = [], []

In [4]:
# Function to fetch images and turn them into numpy arrays
def loadImageData(mainDirectory):
    # Iterating over categories
    for category in os.listdir(mainDirectory):
        # Iterating over images in category
        for imageName in tqdm(os.listdir(os.path.join(mainDirectory + category)), desc = 'Loading Images'):
            # Loading image with pillow
            image = Image.open(os.path.join(mainDirectory, category, imageName))
            # Converting to numpy array
            data = np.asarray(image)

            # Appending label and data to training lists
            imageLabels.append(category)
            imageData.append(data)


In [5]:
# Loading images
loadImageData('./Data/')

In [6]:
# Converting image data to numpy array
imageData = np.array(imageData)

In [7]:
# One-Hot encoding label list
imageLabels = np.array(imageLabels).reshape(-1, 1)

encoder = OneHotEncoder()
imageLabels = encoder.fit_transform(imageLabels).toarray()

In [8]:
# Reshaping data to fit model
# Shape info: (Number of images, Height of image, Width of image, Channels)
# 'Number of images' is -1 so this dimension will be determined by numpy automatically based on the other fixed parameters
# 'Channels' is 1 because we use greyscale images -> only one color channel
imageData = imageData.reshape(-1, 240, 640, 1)

In [9]:
# Assessing shape of training data
print(f"Data Shape: {imageData.shape}")
print(f"Labels Shape: {imageLabels.shape}")

Data Shape: (2000, 240, 640, 1)
Labels Shape: (2000, 10)


In [10]:
# Splitting into train and test sets
trainData, testData, trainLabels, testLabels = train_test_split(imageData, imageLabels, test_size = 0.25, random_state = 111)

### Creating first model

In [11]:
# Creating model called locana (Sanskrit for 'Vision')
locana = Sequential()

In [12]:
# Setting dummy input to avoid OOM error
dummy = tf.zeros((1, 240, 640, 1))
locana._set_inputs(dummy)

In [13]:
# Settin up layers
locana.add(Conv2D(64, kernel_size = 3, activation = 'relu', input_shape = (240, 640, 1)))
locana.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
locana.add(Flatten())
locana.add(Dense(10, activation = 'softmax'))

In [14]:
# Compiling model
locana.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [15]:
# Training model on train data
locana.fit(trainData, trainLabels, batch_size = 8, validation_data = (testData, testLabels), epochs = 3)

Train on 1500 samples, validate on 500 samples
Epoch 1/3
1500/1500 [==============================] - 61s 41ms/sample - loss: 216.0490 - accuracy: 0.9220 - val_loss: 0.2381 - val_accuracy: 0.9620
Epoch 2/3
1500/1500 [==============================] - 56s 38ms/sample - loss: 0.0078 - accuracy: 0.9987 - val_loss: 0.0851 - val_accuracy: 0.9840
Epoch 3/3
1500/1500 [==============================] - 56s 38ms/sample - loss: 9.4740e-04 - accuracy: 1.0000 - val_loss: 0.1102 - val_accuracy: 0.9780


In [ ]:
gc.collect()